In [1]:
import pandas as pd
import glob
from functools import reduce


In [2]:
mojo_df = pd.read_csv('data/mojo.csv', index_col=[0])
metacritic_df = pd.read_csv('data/metacritic.csv', index_col=[0])
imdb_df = pd.read_csv('data/imdb.csv', index_col=[0])
rottentomatoes_df = pd.read_csv('data/rotten_tomatoes.csv', index_col=[0])

In [3]:
rottentomatoes_df.head()

,url,tomato_title,tomato_criticscore,tomato_criticcount,tomato_audiencescore
mojo_title,,,,,
Star Wars: Episode VIII - The Last Jedi,https://rottentomatoes.com/m/star_wars_the_las...,Star Wars: The Last Jedi,90.0,placeholder,42.0
Annabelle: Creation,https://rottentomatoes.com/m/annabelle_creation,Annabelle: Creation,71.0,placeholder,67.0
Murder on the Orient Express,https://rottentomatoes.com/m/murder_on_the_ori...,Murder on the Orient Express,60.0,placeholder,53.0
Daddy's Home 2,https://rottentomatoes.com/m/daddys_home_2,Daddy's Home 2,21.0,placeholder,51.0
Baby Driver,https://rottentomatoes.com/m/baby_driver,Baby Driver,92.0,placeholder,86.0


In [4]:
dfs = [mojo_df, metacritic_df, imdb_df, rottentomatoes_df]
df = reduce(lambda left,right: pd.merge(left,right,on='mojo_title'), dfs)

In [5]:
df.drop_duplicates(subset=['tomato_title'], inplace=True)

In [6]:
df

,domestic_revenue,world_revenue,distributor,opening_revenue,opening_theaters,budget,MPAA,genres,release_days,mc_criticscore,...,mc_audiencecount,imdbpicture,imdbscore,imdbcount,imdb_metacritic,url,tomato_title,tomato_criticscore,tomato_criticcount,tomato_audiencescore
mojo_title,,,,,,,,,,,,,,,,,,,,,
Star Wars: Episode VIII - The Last Jedi,"$620,181,382","$1,332,539,889",Walt Disney Studios Motion Pictures,"$220,009,584","4,232 theaters","$317,000,000",PG-13,"Action, Adventure, Fantasy, Sci-Fi","Dec 15, 2017",84,...,9350,https://m.media-amazon.com/images/M/MV5BMjQ1Mz...,7.0,"563,379",84,https://rottentomatoes.com/m/star_wars_the_las...,Star Wars: The Last Jedi,90.0,placeholder,42.0
Annabelle: Creation,"$102,092,201","$306,515,884",Warner Bros.,"$35,006,404","3,565 theaters","$15,000,000",R,"Horror, Mystery, Thriller","Aug 11, 2017",62,...,279,https://m.media-amazon.com/images/M/MV5BMjA1Mz...,6.5,"112,304",62,https://rottentomatoes.com/m/annabelle_creation,Annabelle: Creation,71.0,placeholder,67.0
Murder on the Orient Express,"$102,826,543","$352,789,811",Twentieth Century Fox,"$28,681,472","3,354 theaters","$55,000,000",PG-13,"Crime, Drama, Mystery","Nov 10, 2017",52,...,408,https://m.media-amazon.com/images/M/MV5BMTAxND...,6.5,"212,816",52,https://rottentomatoes.com/m/murder_on_the_ori...,Murder on the Orient Express,60.0,placeholder,53.0
Daddy's Home 2,"$104,029,443","$180,613,180",Paramount Pictures,"$29,651,193","3,575 theaters","$69,000,000",PG-13,Comedy,"Nov 10, 2017",86,...,1290,https://m.media-amazon.com/images/M/MV5BNDA4Nz...,6.0,"62,252",30,https://rottentomatoes.com/m/daddys_home_2,Daddy's Home 2,21.0,placeholder,51.0
Fifty Shades Darker,"$114,581,250","$381,545,846",Universal Pictures,"$46,607,250","3,714 theaters","$55,000,000",R,"Drama, Romance","Feb 10, 2017",33,...,255,https://m.media-amazon.com/images/M/MV5BMTQ5NT...,4.6,"91,722",33,https://rottentomatoes.com/m/fifty_shades_darker,Fifty Shades Darker,11.0,placeholder,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kong: Skull Island,"$168,052,812","$566,652,812",Warner Bros.,"$61,025,472","3,846 theaters","$185,000,000",PG-13,"Action, Adventure, Fantasy, Sci-Fi","Mar 10, 2017",62,...,755,https://m.media-amazon.com/images/M/MV5BMTUwMz...,6.6,"267,672",62,https://rottentomatoes.com/m/kong_skull_island,Kong: Skull Island,75.0,placeholder,69.0
The Boss Baby,"$175,003,033","$527,965,936",Twentieth Century Fox,"$50,198,902","3,829 theaters",NaN,PG,"Adventure, Animation, Comedy, Family, Fantasy","Mar 31, 2017",50,...,245,https://m.media-amazon.com/images/M/MV5BMTg5Mz...,6.3,"108,453",50,https://rottentomatoes.com/m/the_boss_baby_fam...,The Boss Baby: Family Business,NaN,placeholder,NaN
Logan,"$226,277,068","$619,021,436",Twentieth Century Fox,"$88,411,916","4,071 theaters","$97,000,000",R,"Action, Drama, Sci-Fi, Thriller","Mar 3, 2017",tbd,...,NaN,https://m.media-amazon.com/images/M/MV5BYzc5MT...,8.1,"645,924",77,https://rottentomatoes.com/m/logan_2017,Logan,93.0,placeholder,90.0


Sanity check: do we have any duplicates

In [7]:
boolean = not df["tomato_title"].is_unique      
boolean = df['tomato_title'].duplicated().any() 
boolean

False

Answer: no!

In [8]:
df['domestic_revenue'] = df['domestic_revenue'].str.replace(',', '')
df['domestic_revenue'] = df['domestic_revenue'].str.replace('$', '')
df['domestic_revenue'] = pd.to_numeric(df['domestic_revenue'])

In [9]:
df['domestic_revenue'].apply(type).value_counts()

<class 'int'>    1791
Name: domestic_revenue, dtype: int64

In [12]:
df = df['domestic_revenue'].sort_values(ascending=False)


In [13]:
df.head()

mojo_title
Avengers: Endgame                          858373000
Black Panther                              700059566
Avengers: Infinity War                     678815482
Star Wars: Episode VIII - The Last Jedi    620181382
Incredibles 2                              608581744
Name: domestic_revenue, dtype: int64